In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
import numpy as np
from PIL import ImageOps, Image
from sklearn.model_selection import train_test_split 
from tensorflow import keras
from keras.utils import normalize

image_dir = '/content/drive/MyDrive/BrainDetectionProject/Datasets'

no_tumor_images = os.listdir(image_dir + '/no/')
yes_tumor_images = os.listdir(image_dir + '/yes/')

dataset = []
label = []

# Converting Images from 'jpg' format to 'RGB' for 'no'
for i, image_name in enumerate(no_tumor_images):
    if(image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_dir + '/no/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((64, 64))
        if image.size == (64, 64):
            dataset.append(np.array(image))
            label.append(0)

# Converting Images from 'jpg' format to 'RGB' for 'yes'
for i, image_name in enumerate(yes_tumor_images):
    if(image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_dir + '/yes/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((64, 64))
        if image.size == (64, 64):
            dataset.append(np.array(image))
            label.append(1)

dataset = np.array(dataset)
label = np.array(label)


# Splitting the dataset into training and testing set
x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.2, random_state = 0)

# Normalize the data
x_train = normalize(x_train, axis=2)
x_test = normalize(x_test, axis=2)

# Reshape the data
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 3)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 3)


In [ ]:
# Model Building
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
INPUT_SIZE = 64

model.add(Conv2D(32, (3, 3), input_shape = (INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), kernel_initializer =
                 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size = 16, verbose = 1, epochs = 10, validation_data = (x_test, y_test), shuffle = False)

model.save('BrainTumorDetection10epochs.h5')

Epoch 1/10
150/150 [==============================] - 16s 98ms/step - loss: 0.5470 - accuracy: 0.7188 - val_loss: 0.4723 - val_accuracy: 0.8167
Epoch 2/10
150/150 [==============================] - 14s 93ms/step - loss: 0.4237 - accuracy: 0.8117 - val_loss: 0.3688 - val_accuracy: 0.8367
Epoch 3/10
150/150 [==============================] - 17s 111ms/step - loss: 0.3416 - accuracy: 0.8575 - val_loss: 0.2822 - val_accuracy: 0.8767
Epoch 4/10
150/150 [==============================] - 14s 96ms/step - loss: 0.2497 - accuracy: 0.8983 - val_loss: 0.2172 - val_accuracy: 0.9017
Epoch 5/10
150/150 [==============================] - 14s 95ms/step - loss: 0.1784 - accuracy: 0.9325 - val_loss: 0.1736 - val_accuracy: 0.9300
Epoch 6/10
150/150 [==============================] - 14s 93ms/step - loss: 0.1234 - accuracy: 0.9596 - val_loss: 0.1415 - val_accuracy: 0.9467
Epoch 7/10
150/150 [==============================] - 17s 112ms/step - loss: 0.0916 - accuracy: 0.9725 - val_loss: 0.1070 - val_accurac

In [ ]:
# Load the model
model = keras.models.load_model('/content/BrainTumorDetection10epochs.h5')

# Define a function to predict tumor
def predict_tumor(image_path):
    try:
        # Open the image file with PIL
        image = Image.open(image_path)
        # Resize the image to (64,64)
        image = image.resize((64, 64))
        # Convert the image to numpy array
        image_array = np.array(image)
        # Normalize the image
        image_array = normalize(image_array, axis=2)
        # Reshape the image
        image_array = image_array.reshape(1, image_array.shape[0], image_array.shape[1], 3)
        # Predict the tumor
        prediction = model.predict(image_array)
        if prediction[0][0] == 1:
            return 'Tumor detected'
        else:
            return 'No tumor detected'
    except:
        return 'Invalid image file'

# Test the function on a sample image
image_path = '/content/drive/MyDrive/BrainDetectionProject/Datasets/pred/pred10.jpg'
print(predict_tumor(image_path))